## Introduction to Linear and Discrete Optimization (ADM I): Programming Exercise 2

Technische Universität Berlin, Straße des 17. Juni 135, 10623 Berlin, Deutschland

---

### Homework Group 7

**Mikhail Farber, Tim Rüttinger and Allan A. Zea**

*Disclaimer: The use of this material without permission of the authors is strictly prohibited.*

In [1]:
!pip install picos
!pip install gurobipy
!pip install mosek
!pip install swiglpk
import picos
import numpy as np

Recall the **Scheduling Problem on Parallel Machines** from the Exercise-Videos 1.3a-1.3c. In this programming exercise, we will extend the time-indexed formulation to the case of non-identical machines. In this setting (known as **unrelated machines** in the litterature on scheduling theory), the processing time of a job depends on the machine on which it is executed; Each job has to be executed on exactly one machine, and the execution of job $j\in J=\{0,\ldots,n-1\}$ on machine $i\in M=\{0,\ldots,m-1\}$ requires $p_{i,j}\in\mathbb{N}$ units of processing time. Each machine can process at most one job at a time, and the jobs must be exectuded *non-preemtively*, i.e., jobs cannot be interrupted once started.

**Task:** Given some nonnegative weights $w_j$ ($\forall j\in J$) and some integer processing times $p_{i,j}$ ($\forall i,j \in M\times J$), find a *schedule* minimizing the weighted sum of completion time $\sum_{j\in J} w_j C_j$. Here, a schedule must specify the starting time $S_j\geq 0$ of each job $j\in J$, and the machine $M_j\in M$ on which it is executed.

Generate some unique data for the problem using the following cell, by replacing GROUP by the number of your homework group. Then, formulate the scheduling problem as an IP by adapting the **time-indexed formulation** of the Exercise-Video 1.3c. You are expected to solve both the integer program and its LP-relaxation with the method of your choice: you can either use the PICOS/gurobi framwork that is pre-installed in this notebook, or download the data as a .json file and solve the problem with your favorite solver or interface (ZIMPL/SCIP, cvxpy with any solver, gurobipy interface to gurobi, ...).

Return the solution via ISIS by outputing the solution in the following form: 

``OPT_LP OPT_IP {0: (M0,S0),, 1: (M1,S1), 2: (M2,S2), ... }``,

where
  * ``OPT_LP`` is the value of the optimal objective function in the linear programming relaxation of the problem.
  * ``OPT_IP`` is the value of the optimal objective function of the problem, i.e., $\sum_{j\in J} w_j C_j$
  * ``Sj`` is the starting time of job $j$, for $j=0,...,n-1$
  * ``Mj`` is the machine executing job $j$, for $j=0,...,n-1$.

**Hint** You will need to introduce variables ``x[i,j,t]`` for several triples $(i,j,t)$ to indicate that job $j$ starts (or completes, as you like) on machine $i$ at time $t$. A bound for the time horizon of the problem is $T=\lceil (\frac{n}{m}+1) p_{\max} \rceil$, where $p_{\max}$ is an upper bound for all processing times (*why?*). If x is a dictionary (initialized empty: `x={}`), you can introduce such variables in picos within a for-loop iterating over `(i,j,t)` as follows:
 * ``x[i,j,t] = picos.RealVariable(f'x[{i},{j},{t}]',lower=0,upper=1)`` for a real variable in the interval $[0,1]$;
 * ``x[i,j,t] = picos.BinaryVariable(f'x[{i},{j},{t}]')`` for a binary variable.

After having solved the problem, you can get the value of the optimal variables $x^*$ with `x_star = picos.value(x)`.

In [2]:
import picos

my_group = 7

#function to get a seed for your group # DO NOT CHANGE THIS FUNCTION !
def get_seed_for_my_group(group_number):
  return [1, 3,  5,  6,  9, 10, 11, 14, 15, 16, 17, 19, 20, 21, 23, 24, 26, 27,
          36, 37, 40, 41, 42, 43, 47,49, 51, 56, 58, 59, 60, 63, 65, 67, 71, 77,
          78, 81, 85, 87, 91, 92, 93, 95, 97, 101, 104, 110, 111, 113, 115,
          117, 118][group_number]


#seed the pseudo-random number generator with your group number
seed = get_seed_for_my_group(my_group)
np.random.seed(seed)

#generate the data
n = 14                                  # number of jobs
m = 4                                   # number of machines
pmax = 8                                # processing times are between 1 and pmax
w = np.random.randint(1, 101, n)        # w[j] is the weight of job j
p = np.random.randint(1, pmax + 1, (m, n)) # p[i,j] is the processing time 
                                      #        of job j on machine i         

#uncomment the following block to save the data as a .json file that you can
#use with your favorite software
"""
from google.colab import files
import json

data = {'n':n,
        'm':m,
        'pmax': pmax,
        'p': {i: {j: int(p[i,j]) for j in range(n)} for i in range(m)},
        'w': {j: int(w[j]) for j in range(n)}
        }
with open('ADM1_prog_ex2_data.json','w') as fp:
  json.dump(data, fp, indent=4)

files.download('ADM1_prog_ex2_data.json')
"""

"\nfrom google.colab import files\nimport json\n\ndata = {'n':n,\n        'm':m,\n        'pmax': pmax,\n        'p': {i: {j: int(p[i,j]) for j in range(n)} for i in range(m)},\n        'w': {j: int(w[j]) for j in range(n)}\n        }\nwith open('ADM1_prog_ex2_data.json','w') as fp:\n  json.dump(data, fp, indent=4)\n\nfiles.download('ADM1_prog_ex2_data.json')\n"

You can implement you IP/LP in the following cells

---

## Integer Program (IP) formulation

In [3]:
IP = picos.Problem()

x_IP = {}
T = int((n / m + 1) * pmax) - 2

for i in range(m):
  for j in range(n):
    for t in range(T):
      if t >= p[i,j]:
        x_IP[i,j,t] = picos.BinaryVariable(f'x_IP[{i},{j},{t}]')
      else:
        x_IP[i,j,t] = 0

In [4]:
objective_IP =  picos.sum(w[j] * x_IP[i,j,t] * t for i in range(m) for j in range(n) for t in range(T))
IP.set_objective("min", objective_IP)

print(IP)

Integer Linear Program
  minimize ∑(0, …, 78·x_IP[3,13,33]·33)
  over
    1×1 binary variable x_IP[i,j,k] ∀ (i,j,k) ∈
      zip([0,0,…,3,3],[0,0,…,13,13],[3,4,…,32,33])


In [5]:
for j in range(n):
  IP += (picos.sum([x_IP[i,j,t] for i in range(m) for t in range(p[i,j], T)]) == 1)

# capacity constraints
for i in range(m):
    for t in range(T):
      capacity = picos.sum([x_IP[i,j,s] for j in range(n) for s in range(t, min(t + p[i,j], T))])
      if not isinstance(capacity, int):
        IP += (capacity <= 1)

In [6]:
IP.solve(verbosity = 1, solver = 'gurobi')

            PICOS 2.3.1            
Problem type: Integer Linear Program.
Searching a solution strategy for Gurobi.
Solution strategy:
  1. ExtraOptions
  2. GurobiSolver
Applying ExtraOptions.
Building a Gurobi problem instance.
Restricted license - for non-production use only - expires 2023-10-25
Starting solution search.
Set parameter FeasibilityTol to value 1e-08
Set parameter OptimalityTol to value 1e-08
Set parameter BarQCPConvTol to value 1e-10
Set parameter MIPGapAbs to value 1e-06
-----------------------------------
         Gurobi Optimizer          
-----------------------------------
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 146 rows, 1651 columns and 8810 nonzeros
Model fingerprint: 0xbbda1f85
Variable types: 0 continuous, 1651 integer (1651 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 3e+03]
  Bounds range     [1e

<feasible primal solution (claimed optimal) from gurobi>

In [7]:
x_star_IP = picos.value(x_IP)
print(x_star_IP)

{(0, 0, 0): 0, (0, 0, 1): 0, (0, 0, 2): 0, (0, 0, 3): 0.0, (0, 0, 4): 0.0, (0, 0, 5): 0.0, (0, 0, 6): 0.0, (0, 0, 7): 0.0, (0, 0, 8): 0.0, (0, 0, 9): 0.0, (0, 0, 10): 0.0, (0, 0, 11): 0.0, (0, 0, 12): 0.0, (0, 0, 13): 0.0, (0, 0, 14): 0.0, (0, 0, 15): 0.0, (0, 0, 16): 0.0, (0, 0, 17): 0.0, (0, 0, 18): 0.0, (0, 0, 19): 0.0, (0, 0, 20): 0.0, (0, 0, 21): 0.0, (0, 0, 22): 0.0, (0, 0, 23): 0.0, (0, 0, 24): 0.0, (0, 0, 25): 0.0, (0, 0, 26): 0.0, (0, 0, 27): 0.0, (0, 0, 28): 0.0, (0, 0, 29): 0.0, (0, 0, 30): 0.0, (0, 0, 31): 0.0, (0, 0, 32): 0.0, (0, 0, 33): 0.0, (0, 1, 0): 0, (0, 1, 1): 0, (0, 1, 2): 0, (0, 1, 3): 0.0, (0, 1, 4): 0.0, (0, 1, 5): 0.0, (0, 1, 6): 0.0, (0, 1, 7): 0.0, (0, 1, 8): 0.0, (0, 1, 9): 0.0, (0, 1, 10): 0.0, (0, 1, 11): 0.0, (0, 1, 12): 0.0, (0, 1, 13): 0.0, (0, 1, 14): 0.0, (0, 1, 15): 0.0, (0, 1, 16): 0.0, (0, 1, 17): 0.0, (0, 1, 18): 0.0, (0, 1, 19): 0.0, (0, 1, 20): 0.0, (0, 1, 21): 0.0, (0, 1, 22): 0.0, (0, 1, 23): 0.0, (0, 1, 24): 0.0, (0, 1, 25): 0.0, (0, 1, 26):

In [8]:
IP.value

2376.0

In [9]:
index_IP = 1
solution_IP = {}
for i in range(m):
  for j in range(n):
    for t in range(T):
      if x_star_IP[i,j,t] != 0:
        solution_IP[index_IP] = (j,t)
        index_IP += 1

In [10]:
solution_IP

{1: (2, 2),
 2: (9, 4),
 3: (10, 1),
 4: (12, 8),
 5: (4, 1),
 6: (6, 6),
 7: (8, 2),
 8: (11, 10),
 9: (3, 4),
 10: (5, 9),
 11: (7, 1),
 12: (0, 2),
 13: (1, 8),
 14: (13, 6)}

---

## LP relaxation

In [11]:
LP = picos.Problem()

x_LP = {}
T = int((n / m + 1) * pmax) - 2

for i in range(m):
  for j in range(n):
    for t in range(T):
      if t >= p[i,j]:
        x_LP[i,j,t] = picos.RealVariable(f'x_LP[{i},{j},{t}]', lower = 0, upper = 1)
      else:
        x_LP[i,j,t] = 0

In [12]:
objective_LP =  picos.sum(w[j] * x_LP[i,j,t] * t for i in range(m) for j in range(n) for t in range(T))
LP.set_objective("min", objective_LP)

print(LP)

Linear Program
  minimize ∑(0, …, 78·x_LP[3,13,33]·33)
  over
    1×1 real variable x_LP[i,j,k] (clamped) ∀ (i,j,k) ∈
      zip([0,0,…,3,3],[0,0,…,13,13],[3,4,…,32,33])


In [13]:
for j in range(n):
  LP += (picos.sum([x_LP[i,j,t] for i in range(m) for t in range(p[i,j], T)]) == 1)

# capacity constraints
for i in range(m):
    for t in range(T):
      capacity = picos.sum([x_LP[i,j,s] for j in range(n) for s in range(t, min(t + p[i,j], T))])
      if not isinstance(capacity, int):
        LP += (capacity <= 1)

In [14]:
LP.solve(verbosity = 1, solver = 'gurobi')

            PICOS 2.3.1            
Problem type: Linear Program.
Searching a solution strategy for Gurobi.
Solution strategy:
  1. ExtraOptions
  2. GurobiSolver
Applying ExtraOptions.
Building a Gurobi problem instance.
Starting solution search.
Set parameter FeasibilityTol to value 1e-08
Set parameter OptimalityTol to value 1e-08
Set parameter BarQCPConvTol to value 1e-10
Set parameter MIPGapAbs to value 1e-06
Set parameter QCPDual to value 1
-----------------------------------
         Gurobi Optimizer          
-----------------------------------
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 146 rows, 1651 columns and 8810 nonzeros
Model fingerprint: 0x5e098f85
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.02s
Presolved: 146 rows, 1651 columns, 

<primal feasible solution pair (claimed optimal) from gurobi>

In [15]:
x_star_LP = picos.value(x_LP)
print(x_star_LP)

{(0, 0, 0): 0, (0, 0, 1): 0, (0, 0, 2): 0, (0, 0, 3): 0.0, (0, 0, 4): 0.0, (0, 0, 5): 0.0, (0, 0, 6): 0.0, (0, 0, 7): 0.0, (0, 0, 8): 0.0, (0, 0, 9): 0.0, (0, 0, 10): 0.0, (0, 0, 11): 0.0, (0, 0, 12): 0.0, (0, 0, 13): 0.0, (0, 0, 14): 0.0, (0, 0, 15): 0.0, (0, 0, 16): 0.0, (0, 0, 17): 0.0, (0, 0, 18): 0.0, (0, 0, 19): 0.0, (0, 0, 20): 0.0, (0, 0, 21): 0.0, (0, 0, 22): 0.0, (0, 0, 23): 0.0, (0, 0, 24): 0.0, (0, 0, 25): 0.0, (0, 0, 26): 0.0, (0, 0, 27): 0.0, (0, 0, 28): 0.0, (0, 0, 29): 0.0, (0, 0, 30): 0.0, (0, 0, 31): 0.0, (0, 0, 32): 0.0, (0, 0, 33): 0.0, (0, 1, 0): 0, (0, 1, 1): 0, (0, 1, 2): 0, (0, 1, 3): 0.0, (0, 1, 4): 0.0, (0, 1, 5): 0.0, (0, 1, 6): 0.0, (0, 1, 7): 0.0, (0, 1, 8): 0.0, (0, 1, 9): 0.0, (0, 1, 10): 0.0, (0, 1, 11): 0.0, (0, 1, 12): 0.0, (0, 1, 13): 0.0, (0, 1, 14): 0.0, (0, 1, 15): 0.0, (0, 1, 16): 0.0, (0, 1, 17): 0.0, (0, 1, 18): 0.0, (0, 1, 19): 0.0, (0, 1, 20): 0.0, (0, 1, 21): 0.0, (0, 1, 22): 0.0, (0, 1, 23): 0.0, (0, 1, 24): 0.0, (0, 1, 25): 0.0, (0, 1, 26):

In [16]:
LP.value

2375.0

In [19]:
index_LP = 1
solution_LP = {}
for i in range(m):
  for j in range(n):
    for t in range(T):
      if x_star_LP[i,j,t] != 0:
        print(x_star_LP[i,j,t])
        solution_LP[index_LP] = (j,t)
        print(index_LP,': ',j,t)
        index_LP += 1

0.5
1 :  2 2
0.5
2 :  2 3
0.25
3 :  6 7
0.25
4 :  9 3
0.75
5 :  9 5
1.0
6 :  10 1
0.75
7 :  12 9
0.25
8 :  12 11
1.0
9 :  4 1
0.75
10 :  6 5
0.25
11 :  8 2
0.25
12 :  8 3
0.25
13 :  8 4
0.25
14 :  8 5
1.0
15 :  11 9
1.0
16 :  3 4
1.0
17 :  5 9
1.0
18 :  7 1
1.0
19 :  0 2
1.0
20 :  1 8
1.0
21 :  13 6


In [18]:
solution_LP

{1: (2, 2),
 2: (2, 3),
 3: (6, 7),
 4: (9, 3),
 5: (9, 5),
 6: (10, 1),
 7: (12, 9),
 8: (12, 11),
 9: (4, 1),
 10: (6, 5),
 11: (8, 2),
 12: (8, 3),
 13: (8, 4),
 14: (8, 5),
 15: (11, 9),
 16: (3, 4),
 17: (5, 9),
 18: (7, 1),
 19: (0, 2),
 20: (1, 8),
 21: (13, 6)}